# CKO JAR Revision

### Import modules

In [1]:
import pandas as pd
import numpy as np
import rpy2.rinterface #ggplot tool
from pandas_profiling import ProfileReport
import dask.dataframe as dd
import wrds
import pandasql as ps
import sqlite3

In [ ]:
# ## Review TNIC-3 data

# ### Import TNIC3 data from Hoberg and Philips data library 

# # !wget -P ../2_pipeline/ http://hobergphillips.tuck.dartmouth.edu/idata/tnic3_data.zip
# # !unzip -q ../2_pipeline/tnic3_data.zip -d ../2_pipeline/ && rm ../2_pipeline/tnic3_data.zip

# """
# Hoberg and Philips TNIC3 database
# """
# tnic = pd.read_csv('/Users/ohn0000/Dropbox/Project/cko/0_data/external/tnic3_data.txt', 
#                    delimiter='\t', header=0, index_col=['gvkey1', 'year', 'gvkey2'])
# tnic.dropna(inplace=True)

# ### Subset to 20-closest competitors

# # tnic_industry = tnic.groupby(level=['gvkey1', 'year'])["score"].nlargest(20).reset_index(level=[0,1], drop=True)
# # tnic_industry = tnic_industry.to_frame(name='score')
# # tnic_industry.to_pickle('../2_pipeline/tnic_industry.pkl')
# tnic_industry = pd.read_pickle('../2_pipeline/tnic_industry.pkl')

# ```tnic_industry``` still has firm-years with less than 20 competitors.

# # """
# # Require at least 20 closest competitors
# # """
# # tnicind_sub = tnic.groupby(level=['gvkey1', 'year'])["score"].filter(lambda x: x.size == 20)
# # tnicind_sub = tnicind_sub.to_frame(name='score')

# """
# tnic_industry['gvkey1'] = tnic_industry['gvkey1'].apply(lambda x: str(x).zfill(6))
# tnic_industry['gvkey2'] = tnic_industry['gvkey2'].apply(lambda x: str(x).zfill(6))
# """

# Remeber that _year_ in __tnic_industry__ is the base year for identifying close competitors. Accordingly, _lead1_ is the M&A year and _lead2_ is the year following M&A.

# Readme_tnic3.txt explains that _year_ equals the first four digits of the __compustat__ _datadate_.

# ### Shift years in __tnic_industry__ to get _lead1_ and _lead2_ similarity scores

# tnic_industry.rename(columns={'score':'score_0'}, inplace=True)

# for i in range(1,3):
#     colname = 'score' + '_' + str(i)
#     tnic_industry['score'] = np.NaN
#     tnic_industry.index = tnic_industry.index.set_levels(tnic_industry.index.levels[1] + 1, level=1)
#     tnic_industry.update(tnic)
#     tnic_industry.rename(columns={'score':colname}, inplace=True)

# tnic_industry.reset_index(inplace=True)
# tnic_industry["year"] -= 2
# tnic_industry.set_index(["gvkey1", "year", "gvkey2"], inplace=True)

# tnic_industry.to_pickle('../2_pipeline/tnic_industry.pkl')

# ### Run __*tnic_industry.py*__ on _WRDS_ to update lead scores

# Many of the _lead1_ and _lead2_ values are missing. Grab these values from __TNIC_Advanced__ uploaded on _WRDS_. 

# # !scp ../2_pipeline/tnic_industry.pkl tnic_industry.py $WRDS:~

# ### Download updated __*tnic_industry*__ file from WRDS

# # !scp $WRDS:/scratch/ou/hohn/tnic_ind_update.pkl ../2_pipeline/ 

In [2]:
tnic_industry = pd.read_pickle('../2_pipeline/tnic_ind_update.pkl')

Average TNIC similarity score across 20-closest competitors.  
Remeber that in __TNIC_ALL__ most of the scores equals to zero. The _z\__ might be the more suitable.
- Close pair in t0 not appearing in t1 or t2 is meaningful.
- __BE CAREFUL__ of year 2016 and 2017. __TNIC is available only up to 2017__.

In [3]:
avg_sim = tnic_industry.groupby(level=['gvkey1','year']).mean()
avg_sim = avg_sim.join(tnic_industry.groupby(level=['gvkey1','year']).count().add_prefix("n_"))
avg_sim = avg_sim.join(tnic_industry.fillna(0).groupby(level=['gvkey1','year']).mean().add_prefix("z_"))

In [4]:
len(avg_sim)

109791

### Grab COMPUSTAT _datadate_

Revise here if _avg\_sim_ needs additional COMPUSTAT variables

In [5]:
db = wrds.Connection(wrds_username='yaera')

Loading library list...
Done


In [6]:
comp_query = """
select distinct gvkey, datadate, fyear, indfmt, sale
from comp.funda
where consol = %(consol)s and indfmt in %(indfmt)s 
    and datafmt = %(datafmt)s and popsrc = %(popsrc)s
    and curcd in %(curcd)s
order by gvkey, fyear
"""

parm = {'consol':('C'), 'indfmt' : ('INDL', 'FS'), 'datafmt': ('STD'), 'popsrc' : ('D'), 'curcd' : ('USD', 'CAD')}
comp = db.raw_sql(comp_query, date_cols=['datadate'], params=parm)

In [7]:
comp['fyear'] = comp['fyear'].astype('int16')
comp['year'] = comp['datadate'].dt.year.astype('int16')

comp['gvkey1'] = pd.to_numeric(comp['gvkey']).astype('int64')
comp.drop(columns='gvkey', inplace=True)

comp.set_index(['gvkey1', 'fyear'], inplace=True)

Not sure how TNIC deals fiscal years where fyear are differenct but
year are the same. Assume the lastest datadate within _year_

In [8]:
datadate = comp[~comp.index.duplicated(keep='last')][['datadate', 'year']].reset_index()

In [9]:
avg_sim = pd.merge(avg_sim, datadate, 
                 left_index=True, 
                 right_on=['gvkey1', 'year'], how='left').drop_duplicates(['gvkey1', 'year'], keep='last')

In [10]:
avg_sim = avg_sim[avg_sim['datadate'].notnull()]

TNIC data not missing _datadate_

In [11]:
len(avg_sim)

109757

In [12]:
col = list(avg_sim)
col = col[-4:] + col[:-4]
avg_sim = avg_sim[col].sort_values(['gvkey1', 'year', 'datadate'])

### Grab _permno_

Revise here if _avg\_sim_ needs additional CRSP variables

In [13]:
sql_query= """
select gvkey, liid as iid, lpermno as permno, linkdt, linkenddt
from crsp.ccmxpf_linktable
where linktype in %(type)s and linkprim in %(prim)s and usedflag = 1
"""

parm = {'type':('LU', 'LC'), 'prim':('P', 'C')}
linktable = db.raw_sql(sql_query, date_cols=['linkdt', 'linkenddt'], params=parm)

linktable['gvkey'] = pd.to_numeric(linktable['gvkey'])
linktable['permno'] = pd.to_numeric(linktable['permno']).astype('int64')
linktable['iid'] = linktable['iid'].astype('category')

In [14]:
enddt = pd.to_datetime('2020-01-07 00:00:00')

In [15]:
linktable['linkenddt'] = linktable['linkenddt'].fillna(value = enddt)
linktable['linkenddt'] = linktable['linkenddt'].dt.date

In [16]:
conn = sqlite3.connect(':memory:')
avg_sim.to_sql('avg_sim', conn, index=False)
linktable.to_sql('linktable', conn, index=False)

qry = '''
    select  
        avg_sim.*, linktable.permno
    from
        avg_sim left join linktable on
        avg_sim.datadate between linkdt and linkenddt and avg_sim.gvkey1 = linktable.gvkey
    '''
df = pd.read_sql_query(qry, conn)
df['permno'] = df['permno'].astype('Int64')
df['datadate'] = df['datadate'].astype('datetime64[ns]')

df[df['permno'].isna()].to_sql('df', conn, index=False)

qry = '''
    select
        a.*, b.gvkey1, b.datadate
        from linktable a join df b
        on 
            a.gvkey = b.gvkey1
'''
aug = pd.read_sql_query(qry, conn)

aug.to_sql('aug', conn, index=False)
qry = '''
    select gvkey1, permno, iid, min(linkdt) as linkdt, max(linkenddt) as linkenddt
    from aug
    group by gvkey1, permno, iid
    order by gvkey1, linkdt
'''

df = df.merge(aug[aug['iid'].isin(['01','02'])].rename(columns={'permno':'permno1'})[['gvkey1', 'permno1']], 
         left_on = ['gvkey1'], right_on=['gvkey1'], how='left')
df['permno'] = np.where(df['permno'].isna(), df['permno1'], df['permno'])

In [17]:
df = df.drop_duplicates(['gvkey1', 'year']).drop(columns='permno1')

In [18]:
col = list(df)
col.insert(2, col.pop(col.index('permno')))
df = df[col]

In [19]:
df['dt_s1'] = np.where(df['year'] == 2017, np.NaN, df['score_1'] - df['score_0'])
df['dt_s2'] = np.where(df['year'] == 2017, np.NaN, df['score_2'] - df['score_1'])
df['dt_z1'] = np.where(df['year'] == 2017, np.NaN, df['z_score_1'] - df['z_score_0'])
df['dt_z2'] = np.where(df['year'] >= 2016, np.NaN, df['z_score_2'] - df['z_score_1'])

In [20]:
df.set_index(['gvkey1', 'year'], inplace=True)

## Link SDC to COMPUSTAT

### SDC and Compustat Link File

The link file is from [Michael Ewens](https://github.com/michaelewens/SDC-to-Compustat-Mapping.git). Cite papers below.

 ```
@article{phillips2013r,
  title={R\&D and the Incentives from Merger and Acquisition Activity},
  author={Phillips, Gordon M and Zhdanov, Alexei},
  journal={The Review of Financial Studies},
  volume={26},
  number={1},
  pages={34--78},
  year={2013},
  publisher={Society for Financial Studies}
  }
 ```

```
@article{ewensPetersWang2018,
 title={Acquisition prices and the measurement of intangible capital},
 author={Ewens, Michael and Peters, Ryan and Wang, Sean},
 journal={Working Paper}
 year={2018}
 }
```

SDC processing prohibitively slow. Work on the WRDS cloud using _sdc_link.sas_.

In [27]:
col = ["DealNumber","agvkey","tgvkey","AMANAMES","ACUSIP",
       "APUBLIC","ATTITUDE","FORM","STATUSCODE", "EBITLTM",
       "AMV","ENTVAL","BOOKVALUE",
       "EQVAL","MV","NETASS","NILTM","PCT_CASH","PCT_STK",
       "PCT_OTHER","PCT_UNKNOWN","PR","RANKVAL","SALESLTM",
       "TMANAMES","TNATIONCODE","TPUBLIC","MASTER_CUSIP","TTICKER"]

In [28]:
type = {}
for var in col:
    if var in ["DealNumber","agvkey","tgvkey"]:
        type[var] = 'Int64'
    if var in ["ACUSIP","APUBLIC","ATTITUDE","FORM","STATUSCODE",
               "TNATIONCODE","TPUBLIC","MASTER_CUSIP","TTICKER"]:
        type[var] = 'category'


In [29]:
sdc_link = pd.read_csv('../0_data/external/sdc_gvkey.csv', 
                       header=0,
                       parse_dates=['DATEANN','DATEEFF','DATEFIN'],
                       dtype=type, low_memory=False)

In [30]:
for var in ["EBITLTM", "AMV","ENTVAL","BOOKVALUE",
            "EQVAL","MV","NETASS","NILTM","PR","RANKVAL","SALESLTM"]:
    sdc_link[var] = np.where(sdc_link[var].isin(['nan', 'None', 'P', 'M']), np.NaN,
                             sdc_link[var].str.replace(',',''))
    sdc_link[var] = pd.to_numeric(sdc_link[var]).astype('float')

In [31]:
len(sdc_link)

60316

In [32]:
sdc_link.set_index('DealNumber', inplace=True)

In [33]:
compmna = sdc_link

In [34]:
compmna[['FORM','agvkey']].groupby('FORM').count()

,agvkey
FORM,
Acq. Cert. Asts.,240
Acq. Maj. Int.,2528
Acq. Part. Int.,3
Acq. Rem. Int.,405
Acq. of Assets,42979
Acquisition,2
Merger,14159


Form of the Transaction: 10 codes describing the specific form of the transaction:
- M (MERGER): A combination of business takes place or 100% of the stock of a public or private company is acquired.
- A (ACQUISITION): deal in which 100% of a company is spun off or split off is classified as an acquisition by shareholders.
- AM (ACQ OF MAJORITY INTEREST): the acquiror must have held less than 50% and be seeking to acquire 50% or more, but less than 100% of the target company’s stock.
- AP (ACQ OF PARTIAL INTEREST): deals in which the acquiror holds less than 50% and is seeking to acquire less than 50%, or the acquiror holds over 50% and is seeking less than 100% of the target company’s stock. 
- AR (ACQ OF REMAINING INTEREST): deals in which the acquiror holds over 50% and is seeking to acquire 100% of the target company’s stock.
- AA (ACQ OF ASSETS): deals in which the assets of a company, subsidiary, division, or branch are acquired. This code is used in all transactions when a company is being acquired and the consideration sought is not given.
- AC: (ACQ OF CERTAIN ASSETS): deals in which sources state that “certain assets” of a company, subsidiary, or division are acquired.
- R (RECAPITALIZATION): deals in which a company undergoes a shareholders’ Leveraged recapitalization in which the company issues a special one-time dividend (in the form of cash, debt securities, preferred stock, or assets) allowing shareholders to retain an equity interest in the company.
- B (BUYBACK): deals in which the company buys back its equity securities or securities convertible into equity, either on the open market, through privately negotiated transactions, or through a tender offer. Board authorized repurchases are included.
- EO (EXCHANGE OFFER): deals in which a company offers to exchange new securities for its equity securities outstanding or its securities convertible into equity.

In [38]:
datadate['lagdate'] = datadate.groupby('gvkey1')['datadate'].shift(1) + pd.DateOffset(days=1)

In [39]:
datadate['lagdate'] = np.where(datadate['lagdate'].isna(),
                              datadate['datadate'] - pd.DateOffset(years=1) + pd.DateOffset(days=1),
                              datadate['lagdate'])

In [48]:
list(datadate)

['gvkey1', 'fyear', 'datadate', 'year', 'lagdate']

In [53]:
compmna.to_sql('compmna', conn, index=True, if_exists='replace')
datadate.to_sql('datadate', conn, index = True, if_exists='replace')
qry = '''
    select *
    from 
        (select
                a.*, b.datadate, b.fyear
            from
                compmna a left join datadate b on
                a.agvkey == b.gvkey1 and b.datadate >= a.dateeff
            group by 
                a.DealNumber
        )
        '''
temp1 = pd.read_sql_query(qry, conn)

In [85]:
compmna = temp1[temp1['datedif'] <= 370].drop(columns='datedif')

col = list(compmna)
col.insert(5, col.pop(col.index('datadate')))
col.insert(6, col.pop(col.index('fyear')))
compmna = compmna[col]

In [93]:
compmna.dtypes

DealNumber               int64
agvkey                   int64
tgvkey                 float64
AMANAMES                object
ACUSIP                  object
datadate        datetime64[ns]
fyear                  float64
APUBLIC                 object
ATTITUDE                object
FORM                    object
STATUSCODE              object
DATEANN         datetime64[ns]
DATEEFF         datetime64[ns]
DATEFIN         datetime64[ns]
EBITLTM                float64
AMV                    float64
ENTVAL                 float64
BOOKVALUE              float64
EQVAL                  float64
MV                     float64
NETASS                 float64
NILTM                  float64
PCT_CASH               float64
PCT_STK                float64
PCT_OTHER              float64
PCT_UNKNOWN            float64
PR                     float64
RANKVAL                float64
SALESLTM               float64
TMANAMES                object
TNATIONCODE             object
TPUBLIC                 object
MASTER_C

In [94]:
for col in ['tgvkey']:
    compmna[col] = compmna[col].astype('Int64')
for col in ['agvkey', 'tgvkey', 'ACUSIP', 'APUBLIC', 'ATTITUDE', 'FORM', 'STATUSCODE',
             'TNATIONCODE', 'TPUBLIC', 'MASTER_CUSIP', 'TTICKER']:
    compmna[col] = compmna[col].astype('category')

In [92]:
# date variables
for col in ['datadate', 'DATEANN', 'DATEEFF', 'DATEFIN']:
    compmna[col] = compmna[col].astype('datetime64[ns]')

In [95]:
compmna.set_index('DealNumber', inplace=True)

SDC obsevations with logical (less than 370 day difference from effective date) Compustat _datadate_

In [96]:
len(compmna)

58555

Non-missing target sales

In [108]:
len(compmna[compmna['SALESLTM'].notnull()])

13261

Missing target sales but target _gvkey_ available

In [109]:
len(compmna[(compmna['SALESLTM'].isna()) & (compmna['tgvkey'].notnull())])

6630

In [112]:
len(compmna[(compmna['SALESLTM'].isna()) & (compmna['tgvkey'].isna()) &
   (compmna['RANKVAL'].notnull())])

17842

In [113]:
compmna.dtypes

agvkey                category
tgvkey                category
AMANAMES                object
ACUSIP                category
datadate        datetime64[ns]
fyear                  float64
APUBLIC               category
ATTITUDE              category
FORM                  category
STATUSCODE            category
DATEANN         datetime64[ns]
DATEEFF         datetime64[ns]
DATEFIN         datetime64[ns]
EBITLTM                float64
AMV                    float64
ENTVAL                 float64
BOOKVALUE              float64
EQVAL                  float64
MV                     float64
NETASS                 float64
NILTM                  float64
PCT_CASH               float64
PCT_STK                float64
PCT_OTHER              float64
PCT_UNKNOWN            float64
PR                     float64
RANKVAL                float64
SALESLTM               float64
TMANAMES                object
TNATIONCODE           category
TPUBLIC               category
MASTER_CUSIP          category
TTICKER 

## Construct IV by acquirer's _gvkey_ and _datadate_

### Grab acquirers' lag sales

In [114]:
get_sales = compmna[(compmna['tgvkey'].notnull()) & 
                   (compmna['SALESLTM'].isna())][['agvkey', 'tgvkey', 'fyear']].reset_index(drop=True)

In [115]:
get_sales

,agvkey,tgvkey,fyear
0,17095,18360,1995.0
1,22668,11038,1996.0
2,10719,10719,1995.0
3,15182,24644,1995.0
4,1722,5250,1997.0
...,...,...,...
6625,8247,8247,2016.0
6626,9778,5439,2016.0
6627,2369,1487,2016.0
6628,164633,112112,2017.0


In [ ]:
comp_query = """
select distinct gvkey, datadate, fyear, sale
from comp.funda
where consol = %(consol)s and indfmt in %(indfmt)s 
    and datafmt = %(datafmt)s and popsrc = %(popsrc)s
    and curcd in %(curcd)s and sale is not null
order by gvkey, fyear
"""

parm = {'consol':('C'), 'indfmt' : ('INDL', 'FS'), 'datafmt': ('STD'), 'popsrc' : ('D'), 'curcd' : ('USD', 'CAD')}
sale = db.raw_sql(comp_query, date_cols=['datadate'], params=parm)

sale['fyear'] = sale['fyear'].astype('int16')
sale['gvkey'] = pd.to_numeric(sale['gvkey']).astype('int16')

In [ ]:
sale.drop_duplicates(['gvkey','fyear'], inplace=True)

In [ ]:
sale[sale['gvkey']==112626]

In [ ]:
sale['fyear1'] = sale['fyear'] - 1
lagset = sale[['gvkey', 'fyear', 'sale']].copy()
lagset.rename(columns={'fyear': 'fyear1', 'sale':'l_sale'}, inplace=True)
sale = pd.merge(sale, lagset, on=['gvkey','fyear1'], how='left')

sale.drop(columns=['fyear1'], inplace=True)

In [ ]:
col = list(get_sales)
col = col + ['a_sale']
get_sales = get_sales.merge(sale, 
                            left_on=['agvkey', 'fyear'], right_on=['gvkey', 'fyear'], 
                            how='left').rename(columns={'l_sale':'a_sale'})

get_sales = get_sales[col]
col = col + ['t_sale']
get_sales = get_sales.merge(sale, 
                            left_on=['tgvkey', 'fyear'], right_on=['gvkey', 'fyear'], 
                            how='left').rename(columns={'l_sale':'t_sale'})
get_sales = get_sales[col]

In [ ]:
get_sales['tgvkey'] = np.where(get_sales['tgvkey'] < 0, np.NaN, get_sales['tgvkey'])

In [ ]:
get_sales[get_sales['a_sale'].isna()]

In [ ]:

compmna[compmna['agvkey'] == 112626]

## Private Target Data (Chen 2019)

The _encoding_ option allows proper string imports.

In [ ]:
cw = pd.read_sas('../0_data/manual/CW2019.sas7bdat', format = 'sas7bdat', encoding="utf-8")

In [ ]:
cw['gvkey1'] = pd.to_numeric(cw['gvkey']).astype('int64')

_priv_ dataset is a subset of _compmna_ that will be matched to Ciao-Wei's data.

In [ ]:
priv = sdc[(sdc['RANKVAL'].notnull()) & (sdc['TPUBLIC'] == 'Priv.')]
priv.drop_duplicates(inplace=True)
priv = priv[(priv['DATEEFF'] >= '1997-01-01') & (priv['DATEEFF'] <= '2013-12-31')]
private = priv.compute()

for var in ['DATEANN', 'DATEEFF']:
    private[var] = pd.to_datetime(private[var]).astype('datetime64[ns]')

The code below grabs all but 5 _MASTER_DEAL_NO_ from _private_.

In [ ]:
# merge by dates and names
cw = cw.merge(private[['AMANAMES', 'DATEANN', 'DATEEFF', 'TMANAMES', 'MASTER_DEAL_NO']],
         left_on=['ACQ_NAME', 'ANN', 'EFF', 'TRG_NAME'],
         right_on=['AMANAMES', 'DATEANN', 'DATEEFF', 'TMANAMES'], how='left')
col = list(cw)
col.insert(0, col.pop())
cw = cw[col]

cw.drop(columns=['AMANAMES', 'DATEANN', 'DATEEFF', 'TMANAMES'], inplace=True)

cw.drop_duplicates(inplace=True)

# review dates and names of the missing
missing = private[['MASTER_DEAL_NO', 'AMANAMES', 'DATEANN', 'DATEEFF', 'TMANAMES']].merge(cw[cw['MASTER_DEAL_NO'].isna()][['ANN', 'EFF', 'ACQ_NAME', 'TRG_NAME']],
                                                           how='right',left_on=['DATEANN', 'DATEEFF'], right_on=['ANN', 'EFF'])

missing.drop_duplicates(inplace=True)
missing.reset_index(drop=True, inplace=True)

# manual match
missing = missing.iloc[[0, 1, 5, 16, 24, 26, 28, 29, 39, 44, 53]][['MASTER_DEAL_NO', 'ANN', 'EFF', 'ACQ_NAME', 'TRG_NAME']]

# update MASTER_DEAL_NO
cw = cw.merge(missing, 
         left_on=['ACQ_NAME', 'ANN', 'EFF', 'TRG_NAME'],
         right_on=['ACQ_NAME', 'ANN', 'EFF', 'TRG_NAME'], how='left', suffixes=('','_y'))

cw['MASTER_DEAL_NO'] = np.where(cw['MASTER_DEAL_NO'].isna(), cw['MASTER_DEAL_NO_y'], cw['MASTER_DEAL_NO'])

cw.drop(columns=['MASTER_DEAL_NO_y'], inplace=True)

In [ ]:
col = list(cw)
col.insert(1, col.pop())
cw = cw[col]

In [ ]:
cw.to_sql('compmna', conn, index=False, if_exists='replace')
datadate.to_sql('datadate', conn, index = True, if_exists='replace')
qry = '''
    select  
        a.*, b.datadate
    from
        compmna a join datadate b on
        a.gvkey1 == b.gvkey1 and a.EFF between b.lagdate and b.datadate 
    '''
cw = pd.read_sql_query(qry, conn)

In [ ]:
cw['datadate'] = pd.to_datetime(cw['datadate']).astype('datetime64[ns]')

cw['year'] = cw['datadate'].dt.year

In [ ]:
df.to_sql('avg_sim', conn, index=False, if_exists='replace')
cw.to_sql('cw', conn, index = False, if_exists='replace')
qry = '''
    select  
        a.*
    from
        avg_sim a join (select distinct gvkey1, year from cw) b
        on a.gvkey1 = b.gvkey1 and a.year = b.year
    '''
cw_sim = pd.read_sql_query(qry, conn)

In [ ]:
cw_sim = cw_sim[['gvkey1', 'year', 'dt_s1', 'dt_z1', 'dt_s2', 'dt_z2']]

In [ ]:
tnic_industry.to_sql('tnic', conn, index=True, if_exists='replace')
cw_sim.to_sql('cw_sim', conn, index = False, if_exists='replace')
qry = '''
    select  
        a.gvkey1, a.year, a.gvkey2
    from
        tnic a join (select distinct gvkey1, year from cw_sim) b
        on a.gvkey1 = b.gvkey1 and a.year = b.year
    '''
cw_tnic = pd.read_sql_query(qry, conn)

In [ ]:
cw_tnic = cw_tnic.merge(df[['gvkey1', 'year', 'dt_s1', 'dt_s2', 'dt_z1', 'dt_z2']]
                        , left_on=['gvkey2', 'year'], right_on=['gvkey1', 'year'])

In [ ]:
cw_tnic.drop(columns=['gvkey1_y'], inplace=True)
cw_tnic.rename(columns={'gvkey1_x':'gvkey1'}, inplace=True)

In [ ]:
tnic_avg = cw_tnic.groupby(['gvkey1', 'year']).mean().drop(columns=['gvkey2'])

In [ ]:
cw_sim = cw_sim.merge(tnic_avg, left_on=['gvkey1', 'year'], right_on=['gvkey1', 'year'])

In [ ]:
cw_sim['dt_s1'] = cw_sim['dt_s1_x'] - cw_sim['dt_s1_y']
cw_sim['dt_z1'] = cw_sim['dt_z1_x'] - cw_sim['dt_z1_y']
cw_sim['dt_s2'] = cw_sim['dt_s2_x'] - cw_sim['dt_s2_y']
cw_sim['dt_z2'] = cw_sim['dt_z2_x'] - cw_sim['dt_z2_y'] 
# cw_sim = cw_sim[['gvkey1', 'year', 'dt_s1', 'dt_z1', 'dt_s2', 'dt_z2']]

In [ ]:
cw = cw.merge(cw_sim, left_on=['gvkey1', 'year'], right_on=['gvkey1', 'year'])

In [ ]:
cw.to_stata('/Users/ohn0000/Dropbox/Project/cko/2_pipeline/cw.dta')

### Materiality of M&A

In [ ]:
material = pd.read_csv('/Users/ohn0000/Project/cko/0_data/external/materiality.csv')
material.set_index(["year", "gvkey1"], inplace=True, verify_integrity=True)

Be careful since the _year_ here refers to the M&A firm-year. The _year_ in __avg_sim__ is the year competitors are identified. 

### M&A Disclosure

In [ ]:
disc = pd.read_csv('/Users/ohn0000/Project/cko/0_data/manual/disc.csv', parse_dates=['DATADATE'])
disc['CIK'] = disc['CIK'].apply(lambda x: str(int(x)).zfill(10) if pd.notnull(x) else None)

Disclosure also might need additonal data collection.

In [ ]:
disc.rename(columns={"GVKEY":"gvkey1", "FYEAR":"year"}, inplace=True)
disc.set_index(["year", "gvkey1"], inplace=True, verify_integrity=True)

In [ ]:
manual = disc.join(material)[['DATADATE', 'CIK', 'TGTAT_ACQAT', 'TGTDVAL_ACQAT', 'MD_A', 'PROFORMA']].sort_index()

In [ ]:
import wrds
db = wrds.Connection(wrds_username = "yaera")

In [ ]:
ma_details_desc = db.describe_table('sdc', 'ma_details').sort_values('name')
with pd.option_context('display.max_rows', None):
    print(ma_details_desc)

|     Variable | Description                    |
|:------------:|:-------------------------------|
|bookvalue     |Target Book Value (\$mil)       |
|compete       |Competing Bidder (Y/N)          |
|competecode   |Competing Bid Deal Code         |  
|dateann       |Date Announced                  |
|dateannest    |_dateann_ is estimated (Y/N)    | 
|dateeff       |Date Effective                  | 
|ebitltm       |Target EBIT LTM (\$mil)         |
|pct_cash      |Percentage of consideration paid in cash|
|pct_other|Percentage of consideration paid in other then cash or stock|
|pct_stk|Percentage of consideration paid in stock|
|pct_unknown|Percentage of consideration which is unknown|
|ptincltm|Target Pre-Tax Income LTM (\$mil)|
|salesltm|Target Sales LTM (\$mil)|
|rankval|Ranking Value incl Net Debt of Target (\$mil)|

Run sql query below on _WRDS_

In [ ]:
# import wrds
# sdc_query = """
# select master_deal_no as DealNumber, 
#         bookvalue, 
#         compete, 
#         competecode, 
#         dateann, 
#         dateannest, 
#         dateeff, 
#         ebitltm, 
#         pct_cash,
#         pct_other,
#         pct_stk,
#         pct_unknown,
#         ptincltm,
#         salesltm,
#         rankval
# from sdc.ma_details
# where dateeff is not null 
# """
# # and master_deal_no in %(deal_no)s
# sdc = db.raw_sql(sdc_query, date_cols=['dateann', 'dateeff'])
# sdc.to_pickle('/home/upenn/yaera/sdc.pkl')

In [ ]:
sdc = pd.read_pickle('/Users/ohn0000/Project/cko/0_data/external/sdc.pkl')
sdc.drop_duplicates('dealnumber', inplace = True)
sdc['dealnumber'] = sdc['dealnumber'].astype('int64')

# clear up values and change dtype to 'float'
for column in ['bookvalue', 'ebitltm', 'pct_cash', 'pct_other', 'pct_stk', 'pct_unknown', 'ptincltm', 'salesltm', 'rankval']:
    sdc[column] = sdc[column].apply(lambda x: np.NaN if x == '*********' else (np.NaN if pd.isna(x) else (float(x.replace(',', '')) if isinstance(x, str) else float(x))))
    sdc[column].astype('float16')

In [ ]:
sdc_sub = pd.merge(sdc_link, sdc,
                   left_index=True, right_on='dealnumber').drop('dealnumber', axis='columns')
sdc_sub.index.name = 'dealnumber'

In [ ]:
sdc_sub.sort_values(['agvkey', 'dateeff'], inplace=True)

In [ ]:
sdc_link['agvkey'].count() / sdc['dealnumber'].count()

In [ ]:
sdc_sub['agvkey'].count() / sdc_link['agvkey'].count() 

In [ ]:
sdc_sub.profile_report(style={'full_width':True})

Use __compustat__ _datadate_ and gvkey to link the sdc data to the similarity scores

In [ ]:
import wrds
db = wrds.Connection(wrds_username = 'hohn')

sdc_quary = """
select gvkey, datadate, fyear, cusip,  cik
from comp.funda
where consol = %(consol)s and indfmt in %(indfmt)s and datafmt = %(datafmt)s and popsrc = %(popsrc)s and curcd in %(curcd)s
"""

parm = {'consol':('C'), 'indfmt' : ('INDL', 'FS'), 'datafmt': ('STD'), 'popsrc' : ('D'), 'curcd' : ('USD', 'CAD')}

funda = db.raw_sql(sdc_quary, params = parm, date_cols = ['datadate'])

In [ ]:
funda['start'] = funda['datadate'] - pd.DateOffset(months = 12) + pd.DateOffset(days = 1)
funda['gvkey'] = funda['gvkey'].astype('int64')
funda.set_index('gvkey', inplace=True)

In [ ]:
funda.fyear = funda.fyear.astype('Int16')

In [ ]:
import pandasql as ps

sql_query = '''
select a.*, b.datadate, b.fyear, b.cusip, b.cik
from sdc_sub a left join funda b
on a.agvkey = b.gvkey and a.dateeff between b.start and b.datadate
'''

newdf = ps.sqldf(sql_query, locals())

In [ ]:
col = list(newdf)
for i in range(2, 6):
    col.insert(i, col.pop(-1))
newdf = newdf.loc[:,col]

In [ ]:
for i in ['datadate', 'dateann', 'dateeff']:
    newdf[i] = newdf[i].astype('datetime64[ns]')
    
newdf['year'] = newdf['datadate'].dt.year.astype('Int16')
for i in ['fyear', 'agvkey', 'tgvkey']:
    newdf[i] = newdf[i].astype('Int64')

In [ ]:
col = list(newdf)
col.insert(col.index('datadate'), col.pop(col.index('year')))
newdf = newdf.loc[:,col]

In [ ]:
newdf = newdf.drop_duplicates(subset='dealnumber')

In [ ]:
newdf[newdf['agvkey'].notnull()]

In [ ]:
newdf['rankval'].count()

18994 observations with non-missing _rankval_

In [ ]:
newdf['salesltm'].count()

8055 observations with non-missing _salesltm_

In [ ]:
np.sum(newdf['rankval'].notnull() & newdf['salesltm'].notnull())

6445 observations with both _rankval_ and _salesltm_ available

## Append similarity score between acquirer and target

In [ ]:
upload = newdf[newdf['agvkey'].notnull() & newdf['tgvkey'].notnull() & newdf['year'].notnull()][['agvkey', 'tgvkey', 'year']].rename(columns={'agvkey':'gvkey1', 'tgvkey':'gvkey2'})
upload.to_csv('/Users/ohn0000/Project/cko/2_pipeline/upload.csv', index=False)
!scp /Users/ohn0000/Project/cko/2_pipeline/upload.csv $WRDS:/scratch/ou/hohn

Run this on wrds server. The __TNIC_All__ files should be uploaded in scratch beforehand.

In [ ]:
"""
The server killed the previous code that joins after combines all files. The current code instead loop over the files.
"""
# !cd /scratch/ou/hohn/TNIC_AllPairsDistrib
# !cat tnicall1996.txt > tnicall_combined.txt
# !for file in tnicall{1997..2017}.txt; do sed '1d' $file >> tnicall_combined.txt; done
# !cd ~


"""
atsim.py
"""


In [ ]:
!scp atsim.py $WRDS:~

In [ ]:
!scp $WRDS:/scratch/ou/hohn/atsim.csv /Users/ohn0000/Project/cko/2_pipeline/

In [ ]:
col = list(newdf)
col.insert(col.index('bookvalue'), col.pop(col.index('atsim')))
newdf = newdf.loc[:,col]

## IV candidates

The materiality measure based on deal value will be the last resort for the IV.   
Alternatively, 2SLS using multiple IVs is feasible.

Candidates
* Max deal value
* Sum deal value
* Datedif between _dateeff_ and _datadate_
    * _dateeff_ of the first M&A
    * _dateeff_ of the largest M&A
    * weighted average of _dateeff_ 

## Cross-sections
* Similarity between acquirer and target 
    - Relation stronger in diversifying
    - Could be more of a U-shaped relation, i.e., competitors don't follow when you move far enough
* Average value of pre-similarities between acquirer and close competitors 
    - Prediction not clear
* M&A performance during the completed firm-year
    - Relation stronger when M&A was more successful <-> how do we define success of an M&A?
* Number of close competitors of the target
    - Potential targets are candidates of future mergers
* How many competitors were there initially?
    - The size of the TNIC industry